In [1]:
from mxnet import nd,gluon,init,autograd, metric

from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets,transforms

import matplotlib.pyplot as plt
from time import time


# Get data

In [18]:
mainst_train = datasets.FashionMNIST(train=True)
X,y = mainst_train[0]

print("Shape : {} and Datatype : {}".format(X.shape,X.dtype))
print("Number of Intiger : ",len(mainst_train))

Shape : (28, 28, 1) and Datatype : <class 'numpy.uint8'>
Number of Intiger :  60000


In [3]:
# sample_idx = 123
# sample_data, sample_label = mainst_train[sample_idx]
# plt.imshow(sample_data.asnumpy())

# Transform Data

In [19]:
transformer = transforms.Compose([transforms.ToTensor(),transforms.Normalize(0.13,0.31)])

mainst_train = mainst_train.transform_first(transformer)

# Data Loading

In [20]:
batch_size = 256

train_data = gluon.data.DataLoader(mainst_train,batch_size=batch_size,shuffle=True,num_workers=4)

In [21]:
for data, label in train_data:
    print(data.shape,label.shape)
    break

(256, 1, 28, 28) (256,)


# Define the model

In [22]:
net = nn.Sequential()
with net.name_scope():
    net.add(
        nn.Conv2D(channels=6,kernel_size=5,activation='relu'),
        nn.MaxPool2D(pool_size=2,strides=2),
        nn.Conv2D(channels=16,kernel_size=3,activation='relu'),
        nn.MaxPool2D(pool_size=2,strides=2),
        nn.Flatten(),
        nn.Dense(120,activation='relu'),
        nn.Dense(84,activation='relu'),
        nn.Dense(10)
    )
    
net.initialize(init=init.Xavier())

# Loss

In [23]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

# Matrix Accuracy

In [24]:
train_acc = metric.Accuracy()

# Optimizer 

In [25]:
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.1})

# Training Loop

In [28]:
for epoch in range(10):
    train_loss = 0.
    tic = time()
    
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output,label)
        loss.backward()
        
        trainer.step(batch_size)
        
        train_loss += loss.mean().asscalar()
        train_acc.update(label,output)
        
    print("Epoch: %d loss: %.3f Acc: %.3f Pref: %.1f img/sec"%(epoch,train_loss/len(train_data),train_acc.get()[1],len(mainst_train)/(time()-tic)))

net.save_parameters('trained_params.params')

Epoch: 0 loss: 0.836 Acc: 0.692 Pref: 3590.7 img/sec
Epoch: 1 loss: 0.470 Acc: 0.759 Pref: 3595.8 img/sec
Epoch: 2 loss: 0.399 Acc: 0.790 Pref: 3581.8 img/sec
Epoch: 3 loss: 0.361 Acc: 0.810 Pref: 3603.0 img/sec
Epoch: 4 loss: 0.334 Acc: 0.823 Pref: 3511.5 img/sec
Epoch: 5 loss: 0.315 Acc: 0.834 Pref: 3594.6 img/sec
Epoch: 6 loss: 0.301 Acc: 0.841 Pref: 3573.4 img/sec
Epoch: 7 loss: 0.289 Acc: 0.848 Pref: 3612.3 img/sec
Epoch: 8 loss: 0.280 Acc: 0.853 Pref: 3598.7 img/sec
Epoch: 9 loss: 0.270 Acc: 0.858 Pref: 3569.8 img/sec


# Exit